In [1]:

import numpy as np
import math

#global variables
alpha = [104, 128, 135, 139, 150, 153, 162, 168, 195, 198]
beta = [9, 8, 7, 7, 6, 6, 5, 2, 1, 1]
R2 = 68644
x_init = [0] * 10

#compute the function
def compute_f(alpha,x):
    return np.matmul(alpha, x)

#compute the constraint
def compute_g(beta,x):
    g = 0
    for i in range(len(beta)):
        g+=beta[i]*(x[i]**2)
    return g

#compute sum(alpa[i]^2/beta[i]) for substituting in the formula
def compute_sum():
    s = 0
    for i in range(len(alpha)):
        s += ((alpha[i] ** 2) / beta[i])
    return s

#computes updated g(x)'s RHS and updated summation of (alpa[i]^2/beta[i])
# after partial assign to calculate the real solutions for the relaxed variables
def compute_real_solutions(indx=0,x = x_init):
    s = compute_sum()
    val = x
    G = R2
    for i in range(indx):
        G -= beta[i]*x[i]*x[i]
        s -= alpha[i]*alpha[i]/beta[i]
    s = (s ** 0.5)
    for i in range(indx, len(alpha)):
        val[i] = (alpha[i] * (G ** 0.5)) / (beta[i] * s)
    return val

#integer solution for the previous
def compute_integer_solutions(indx=0,x = x_init):
    s = compute_sum()
    val = x
    G = R2
    for i in range(indx):
        G -= beta[i]*x[i]*x[i]
        s -= alpha[i]*alpha[i]/beta[i]
    s = (s ** 0.5)
    for i in range(indx, len(alpha)):
        val[i] = math.floor((alpha[i] * (G ** 0.5)) / (beta[i] * s))
    return val

# upper bound value of the function
def compute_upper_bound(x):
    return compute_f(alpha,x)

# lower bound value of the function
def compute_lower_bound(x):
    x = [math.floor(i) for i in x]
    return compute_f(alpha,x)

# maximum value of each of the x wrt to the constraint
def compute_max_variable_value(indx=0,val=x_init):
    G = R2
    for i in range(indx):
        G -= beta[i]*val[i]*val[i]
    return (math.floor(math.sqrt(G/beta[indx])))

#partial assignment of variable and relaxation
def partial_assign(indx=0,val = x_init):
    val[indx] = compute_max_variable_value(indx,val)
    val = compute_real_solutions(indx + 1,val)
    return val

#computing new x with gradient descent
def compute_newx(x,gd):
    return np.add(x,gd)

#compute gradient descent
def compute_gd(alpha,beta,step):
    gd = []
    for i in range(len(beta)):
        gd.append(((step*alpha[i])/beta[i]))
    return gd

#greedy gradient descent
def greedy():
    x = [0]*10
    lower_bound = compute_lower_bound(compute_integer_solutions())
    step = 0.005
    solution=[]
    while (True):
        # print(compute_f(alpha,x))
        # print(compute_g(beta,x).sum())
        # print(x)
        last = x
        x = compute_newx(x,compute_gd(alpha,beta,step))
        # print(x)
        
        if(compute_f(alpha,x) >= lower_bound):
            solution = last
            break
        if (compute_g(beta,x)<=R2):
            solution = last
        else:
            break
        
    for i in range(len(solution)) :
        solution[i] = math.floor(solution[i])
    print(solution)
    return solution
        
    
def branch_bound():
    indx = 0
    ctr = 0
    complete_solutions_ctr = 0
    solutions = []
    lower_bound = compute_lower_bound(compute_real_solutions())
    val = partial_assign(indx)
    flag = True
    while True:
        ctr += 1
        local_upper_bound = compute_upper_bound(val)
        
        if local_upper_bound < lower_bound:
            
            val[indx] -= 1
            if val[0] < 0:
                #eventhough the constraint might get satisifed, negative val wont give max val for the function.
                break
            
            val = compute_real_solutions(indx + 1,val)
            if val[indx] == 0:
                if indx == 0 and flag:
                #flag keeps track if the the value is init 0 or calculated as 0.
                    flag = True
                    indx = 0
                else:
                    indx -= 1
            continue
        
        elif indx + 1 < len(val):
            #relaxation 
            indx += 1
            val = partial_assign(indx,val)
            
        else:
            
            if local_upper_bound == lower_bound:
                #adding to the list of possible solutions
                updated_val = val.copy()
                solutions.append(updated_val)
                # print(solutions)
            else:
                #first solution after the new upper bound is updated.
                lower_bound = local_upper_bound
                updated_val = val.copy()
                complete_solutions_ctr+=len(solutions)
                solutions = [updated_val]
            if updated_val[0] == 0:
                break
            
            val[indx] -= 1
            
    print("Solutions:",solutions)
    print("Number of optimal solutions:",len(solutions))
    # print(lower_bound)
    print("Number of possible solutions checked",ctr)
    # print(local_upper_bound)
    print("Number of complete variable assignments",complete_solutions_ctr)
    print("Maximum value of solution",lower_bound)
    
def main():
    print("The sum:",compute_sum()**0.5)
    real_solution = compute_real_solutions()
    print("The real value solutions:",real_solution)
    print("The upper bound",compute_upper_bound(real_solution))
    print("The lower bound",compute_lower_bound(real_solution))

    print("Greedy solution:")
    print(compute_f(alpha,greedy()))
    print("Branch and bound solution:")
    branch_bound()
    
if __name__ == "__main__":
    main()


The sum: 335.9386730691959
The real value solutions: [9.0122269278951, 12.4784680540086, 15.041010600813937, 15.486670174171387, 19.497606334388436, 19.887558461076203, 25.268897809367417, 65.51195728354514, 152.0813294082298, 154.42104216835642]
The upper bound 88015.93234412931
The lower bound 87441
Greedy solution:
[  8.  12.  14.  15.  19.  19.  24.  64. 150. 152.]
86086.0
Branch and bound solution:
Solutions: [[9, 13, 15, 16, 20, 20, 25, 65, 152, 154]]
Number of optimal solutions: 1
Number of possible solutions checked 4957051
Number of complete variable assignments 423
